# Script pour fusionner les données capta 

## Si il y a un problème de librairies non installées
Tapez !pip install librairie

Ex : 
```bash
!pip install pandas
```

In [1]:
import pandas as pd
from datetime import datetime, timedelta, time

## 1 - Import du fichier de données des PM, qui est placé dans le dossier trajet/nom_fichierPM

In [ ]:
dfPM = pd.read_csv('trajet/nom_fichierCM', skiprows=4, delimiter='\t', header=0, encoding = 'unicode_escape', engine='python', skipfooter=1 )

In [ ]:
# Pour afficher le tableau, executer cette cellul
dfPM

## 2 - Import du fichier de données des CM, placé dans le dossier trajet/nom_fichierCM

In [ ]:
dfCM = pd.read_csv('trajet/nom_fichierCM', delimiter='\t', header=0, encoding = 'unicode_escape', engine='python')

In [ ]:
# Pour afficher le tableau, executer cette cellul
dfCM

## 3 - Fonction qui fusionne les 2 tableaux PM et CM. Les paramètres de la fonction sont :
- PM : tableau PM
- CM : tableau CM
- index_beginning_PM : index de la première ligne à fusionner (0 en principe) du tableau PM
- index_end_PM : index de la derniere ligne à fusionner du tableau CM (en principe la derniere, c'est à dire dfPM.shape[0] en principe
- index_beginning_CM : index de la première ligne à prendre en compte sur le tableau CM (à regarder dans le tableau)
- index_end_CM : index de la dernière ligne à prendre en compte du tableau CM (dernière à priori : dfCM.shape[0])

In [ ]:
def mergeDF_PM_CM(PM, CM, index_beginning_PM, index_end_PM, index_beginning_CM, index_end_CM):
    dfPMs = pd.concat([PM.iloc[index_beginning_PM:index_end_PM].reset_index(), CM.iloc[index_beginning_CM:index_end_CM].reset_index()], axis = 1)
    return dfPMs

In [ ]:
# On utilise cette fonction pour les fusionner : (exemple ici)
dfPMs = mergeDF_PM_CM(PM, CM, 0, dfPM.shape[0], 3, dfCM.shape[0])

## 4 - Import du fichier de données météo

Fichier placé ici : trajet/nom_fichier_meteo

### 4.1 Si le fichier météo est potenvol

In [ ]:
dfWeather = pd.read_csv('trajet/nom_fichier_potenvol', skiprows=14, delimiter=';', header=0)
# Remove space at the end of heure
dfWeather['heure'] = dfWeather['heure'].apply(lambda x: x.strip())

In [ ]:
# Pour voir le tableau
dfWeather

### 4.2 - Si le fichier météo vient de termite

In [ ]:
dfWeather = pd.read_csv('trajet/nom_fichier_meteo', skiprows=4, header=None)
# On remplit le nom des colonnes
dfWeather.set_axis(['Node letter', 'Wind direction', 'Wind speed', 'Pressure', 'Humidity', 'Temperature', 'Dewpoint', 'PRT', 'An I/P 1', 'An I/P 2', 'DigI/P', 'Supply Voltage', 'Status Code', 'End character' ], axis=1, inplace=True)

In [ ]:
# on va chercher l'heure directement dans le fichier termite
with open("trajet/nom_fichier_meteo", 'r') as f:
    lines = f.readlines()
    startTime = lines[2].split(" ")[-2]
    
# On rajoute la colonne heure dans le tableau
dfWeather['heure'] = datetime.strptime(startTime, "%H:%M:%S")

In [ ]:
# On crée une nouvelle colonne d'heure en rajoutant 1sec a chaque ligne
results = []
k=0
for _, row in df.iterrows():
    result = datetime.strptime(startTime, "%H:%M:%S") + timedelta(seconds=k)
    k += 1
    results.append(result)
df['time2'] = results
df['heure'] = df['time2'].dt.time
df.drop(['time2'], inplace=True, axis=1)

## 5 - On fusionne le fichier météo avec le tableau des PM-CM

In [ ]:
dfPMsWithWeather = dfPMs.merge(right=dfWeather, how='left', left_on='time beginning', right_on='heure')

## 6 - Import des données de géoloc

### 6.1 - Données au format gpx

On utilise le script gpx_to_csv.py pour les transformer en .csv

In [3]:
!python gpx_to_csv.py nom_fichier_gpx 

Launching scriptgpx_to_csv.py
INFO : Launching merge
Traceback (most recent call last):
  File "gpx_to_csv.py", line 28, in <module>
    main((sys.argv[1:]))
  File "gpx_to_csv.py", line 15, in main
    df = Converter(input_file='SO_25-01-22_Pied_3_Aller_Geoloc.gpx').gpx_to_dataframe()
  File "/Users/mathis/Desktop/Centrale_Nantes/EI4/Cours/MINT/captation/venv/lib/python3.7/site-packages/gpx_converter/base.py", line 22, in __init__
    raise Exception(f"The file {input_file} does not exist.")
Exception: The file SO_25-01-22_Pied_3_Aller_Geoloc.gpx does not exist.


Un fichier est créé dans datagpxtocsv/nom_fichier_gpx.csv. On peut donc le charger comme il faut :


In [ ]:
dfGeoloc = pd.read_csv('datagpxtocsv/nom_fichier.csv', index=False)

In [ ]:
dfGeoloc.rename(columns={"time": "Timestamp"}, inplace=True)

### 6.2 Sinon on l'importe direct

In [ ]:
dfGeoloc = pd.read_csv('nom_du_fichier', delimiter = ',')

On crée une fonction pour décaler l'heure de début de la géoloc

In [ ]:
def createColTimeGeoloc(df, hour, minutes, seconds):
    df['timeTmp'] = df['Timestamp'].apply(
        lambda x:
        datetime.fromisoformat(x) +
        timedelta(hours=hour, minutes=minutes, seconds=seconds)
    )
    df['time'] = df['timeTmp'].apply(lambda x: str(x).split(" ")[1].split("+")[0])
    return df

In [ ]:
dfGeoloc = createColTimeGeoloc(dfGeoloc, 1, 10, 39) # on décale d'1h, 10min et 39sec

## 7- On fusionne ce nouveau tableau pour obtenir le final

In [ ]:
dfFinal = dfPMsWithWeather.merge(right=dfGeoloc, how='left', left_on='time beginning', right_on='time')

## 8 - On supprime les colonnes inutiles

In [ ]:
dfFinal2 = dfFinal.drop(['Unnamed: 9', 'Unnamed: 15', 'Timestamp', 'timeTmp', 'time_y', 'Unnamed: 7', 'time_x', 'Chrono', 'heure', 'date end', 'relative time [s]', 'index', 'date', 'POM [Km).', 'CN[P/cm3)', 'PMTot[µm)', 'PM1[µm)', 'PM2.5[µm)', 'PM4[µm)', 'PM10[µm)'], axis=1)

## 9 - On rentre une nouvelle colonne du trajet en chaines de caracteres

In [ ]:
dfFinal2['trajet'] = 'Erdre_aller_0712'

## 10 - On le sauvegarde dans un fichier csv 

In [ ]:
dfFinal2.to_csv('nom_du_nouveau_fichier_qu_on_cree.csv', index=False)